<a href="https://colab.research.google.com/github/aktasceren/NumericalAnalysis/blob/main/cse317_hw02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from sympy import diff, sin, exp
import numpy as np
import math
from sympy.abc import * 
import pandas as pd
import matplotlib.pyplot as plt

Question 1

In [8]:
x, a, h = symbols("x a h")

In [9]:
x_coordinates = np.array([-2, 0, 1, 3]) # x coordinates
y_coordinates = np.array([8, 4, 2, -2]) # f(x)

In [10]:
def NDD(x_list, y_list, degree):
  n = len(y_list)
  # coefficients table
  table = np.zeros([n, degree+2])
  # set first column as x 
  table[::,0] = x_list
  # set second column as y 
  table[::,1] = y_list 
  k = 0
  for j in range(2,degree+2):
    for i in range(j-1, n):
      table[i][j] = (table[i][j-1] - table[k][j-1]) / (table[i][0] - table[k][0])
    k+=1
      
  # initialize y for given degree 
  y_degree = 0
  for i in range(degree + 1):  
    xs = 1    
    for j in range(i):
      # (x - x0), (x - x0)*(x - x1)...
      xs  *= (x - table[j][0])

    y_degree += xs*table[i][i+1] 
  return y_degree

In [11]:
NDD(x_coordinates, y_coordinates, 3)

4.0 - 2.0*x

Question 2

In [12]:
#derivative of function
def der(expr, n):
    if n > 0:
        return der(diff(expr), n-1)
    return expr

In [13]:
def newton_raphson(f, x0):
  # for x values
  xs = [x0]
  derivative_of_func = der((f), 1)
  i = 0
  # the roots until the result equal to zero
  while True:
    x0 = xs[i]
    x_next = x0 - float(f.subs([(x, x0)])/derivative_of_func.subs([(x, x0)]))
    xs.append(x_next)
    result_of_func = f.subs([(x, x_next)])
    round_of_result = round(result_of_func, 10)
    i += 1
    if round_of_result == 0:
      print("result:" + "\n{:.4f}".format(result_of_func))
      print("root:")
      return round(x_next, 4)      
      break
  

In [14]:
func = 3*x*x*x + x**2 - x - 5

In [15]:
newton_raphson(func, -1)

result:
0.0000
root:


1.1697

Question 3

In [16]:
def finite_difference(x, order, df):
  labels =["f\'(x)", "f\'\'(x)", "f\'\'\'(x)", "f⁽\N{SUPERSCRIPT FOUR}⁾(x)"]

  # coefficient tables
  df_forward_difference = pd.DataFrame(np.array([[2*h, -3, 4, -1, None, None, None], [h**2, 2, -5, 4, -1, None, None], [2*h**3, -5, 18, -24, 14, -3, None], [h**4, 3, -14, 26, -24, 11, -2]]), index = labels,
                                      columns = ['Coefficients of derivatives', 'f(x)', 'f(x+h)', 'f(x+2h)', 'f(x+3h)', 'f(x+4h)', 'f(x+5h)'])
  
  df_backward_difference = pd.DataFrame(np.array([[2*h, None, None, None, 1, -4, 3], [h**2, None, None, -1,  4, -5, 2], [2*h**3, None, 3, -14, 24, -18, 5], [h**4, -2, 11, -24, 26, -14, 3]]), index = labels,
                                     columns = ['Coefficients of derivatives', 'f(x-5h)', 'f(x-4h)', 'f(x-3h)', 'f(x-2h)', 'f(x-h)', 'f(x)'])
  
  df_central_difference = pd.DataFrame(np.array([[2*h, None, -1, 0, 1, None], [h**2, None, 1, -2, 1, None], [2*h**3, -1, 2,  0, -2,  1], [h**4, 1, -4, 6, -4, 1]]), index = labels, 
                                  columns= ['Coefficients of derivatives', 'f(x-2h)', 'f(x-h)', 'f(x)','f(x+h)', 'f(x+2h)'])
  
  # index of x in data table
  index = df.columns.get_loc(x)

  # h in the formula
  difference = df.columns[index] - df.columns[index - 1]

  # derivative order
  der = labels[order - 1]

  # coefficient list according to derivative order
  get_coeff_FD = df_forward_difference.loc[der].tolist()
  get_coeff_BD = df_backward_difference.loc[der].tolist()
  get_coeff_CD = df_central_difference.loc[der].tolist()

  # initialize ?
  forward_difference, backward_difference, central_difference = 0, 0, 0

  # calculations for Forward Difference Approximation
  for i in range(1, len(get_coeff_FD) - 1):
    if not (0 <= index + i - 1 < len(df.iloc[0])) and get_coeff_FD[i] is not None:
      forward_difference = "There is not enough data for calculation for Forward Finite Difference Approximation"
      break
    else:
      if get_coeff_FD[i] is None:
        pass
      else:
        forward_difference += get_coeff_FD[i]*df.iloc[0][df.columns[index + i - 1]]

  # y value for the derivative of selected order 
  if type(forward_difference) is not str:
    forward_difference = forward_difference / get_coeff_FD[0].subs([(h, difference)])

  # calculations for Backward Difference Approximations
  for i in range(len(get_coeff_BD) - 1, 1, -1):
    if not (0 <= i - index - 2 < len(df.iloc[0])) and get_coeff_BD[i] is not None:
      backward_difference = "There is not enough data for calculation for Backward Finite Difference Approximation"
      break
    else:
      if get_coeff_BD[i] is None:
        pass
      else:
        backward_difference += get_coeff_BD[i]*df.iloc[0][df.columns[i - index - 2]]

  # y value for the derivative of selected order
  if type(backward_difference) is not str:
    backward_difference = backward_difference / get_coeff_FD[0].subs([(h, difference)])

  for i in range(len(get_coeff_CD) - 1, 1, -1):
    if not (0 <= i - index + 1 < len(df.iloc[0])) and get_coeff_CD[i] is not None:
      central_difference = "There is not enough data for calculation for Central Finite Difference Approximation"
      break
    else:
      if get_coeff_CD[i] is None:
        pass
      else:
        central_difference += get_coeff_CD[i]*df.iloc[0][df.columns[i - index + 1]]
  
  # y value for the derivative of selected order
  if type(central_difference) is not str:
    central_difference = central_difference / get_coeff_FD[0].subs([(h, difference)])

  print("Finite Difference Approximations Result of O(h\N{SUPERSCRIPT TWO}) to compute " + der + " where x = " + str(x) + ":")
  print(forward_difference)
  print(backward_difference)
  print(central_difference)

  # display coefficient tables
  # print("\n            \033[1mCoefficients of Forward Finite Difference Approximations")
  # display(df_forward_difference)
  # print("\n            \033[1mCoefficients of Backward Finite Difference Approximations")
  # display(df_backward_difference)
  # print("\n          \033[1mCoefficients of Central Finite Difference Approximations")
  # display(df_central_difference)

In [17]:
data = pd.DataFrame(np.array([[0.85866, 0.86289, 0.86710, 0.87129]]), columns = [2.36, 2.37, 2.38, 2.39])

In [18]:
finite_difference(2.38, 1, data)

Finite Difference Approximations Result of O(h²) to compute f'(x) where x = 2.38:
There is not enough data for calculation for Forward Finite Difference Approximation
0.420000000000007
0.420000000000007


In [19]:
finite_difference(2.38, 2, data)

Finite Difference Approximations Result of O(h²) to compute f''(x) where x = 2.38:
There is not enough data for calculation for Forward Finite Difference Approximation
There is not enough data for calculation for Backward Finite Difference Approximation
-0.199999999999098


Question 4

In [20]:
def simpsons_rule(f, panel, a, b):
  iteration = int(panel / 2)
  h = (b-a)/panel
  f_a = f.subs([(x, a)])
  f_b = f.subs([(x, b)])
  func_subs = (f_a + 4*f.subs([(x, h)]) +f_b)
  j = 2
  for i in range(iteration-1): 
    func_subs += 2*f.subs([(x, j*h)]) + 4*f.subs([(x, (j+1)*h)])
    j += 2
  I = func_subs*h/3
  return I

In [21]:
# two panels
simpsons_rule(x**2, 2, 0, 1)

0.333333333333333

In [22]:
# four panels
simpsons_rule(x**2, 4, 0, 1)

0.333333333333333

In [23]:
# six panels
simpsons_rule(x**2, 6, 0, 1)

0.333333333333333